In [1]:
#going to unrotate all the imaging tiff files under a folder
#save all of them into a folder named UnrotTiff with the 
#same file name but add _unrot at the end

In [1]:
import glob
import os
import numpy as np
from PIL import Image
#import utils_image from the parent folder
import sys
sys.path.append("..")
from utils_image import cropLargestRecT
from scanimagetiffio import SITiffIO

parent_folder = "/home/zilong/Desktop/2D2P/Data/183_25072023/"

In [2]:
# read the rotation center from the circlecenter txt file
circlecenterfilename = parent_folder+"DP_exp/circlecenter.txt"
with open(circlecenterfilename, "r") as f:
    # read the last row
    last_line = f.readlines()[-1]
    # assign the x and y coordinates to self.rotx and self.roty
    rotx = float(last_line.split()[0])
    roty = float(last_line.split()[1])

In [24]:
#get all the tiff files under the parent folder with an extension of .tif
tifffiles = glob.glob(parent_folder + "/*.tif")
#remove tifffiles with key words "stack"
tifffiles = [x for x in tifffiles if "stack" not in x]
#remove tifffiles with key words "00010" and "00012"
tifffiles = [x for x in tifffiles if "00010" not in x]
tifffiles = [x for x in tifffiles if "00012" not in x]
#sort the tifffiles by the number in the file name
tifffiles.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))

#get all the RElog files under the parent folder begin with 'RE' and with an extension of .txt
RElogfiles = glob.glob(parent_folder + "/RE*.txt")
#remove RElogfiles with key words "stack", "00010" and "00012"
RElogfiles = [x for x in RElogfiles if "stack" not in x]
RElogfiles = [x for x in RElogfiles if "00010" not in x]
RElogfiles = [x for x in RElogfiles if "00012" not in x]


#pair the tiff files and RElog files together which share the same key word
allfiles = []
for tifffile in tifffiles:
    #extract the key word from the tifffile
    #/home/zilong/Desktop/2D2P/Data/183_25072023/25072023_00005.tif' then extract 00005
    key = tifffile.split("/")[-1].split(".")[0].split("_")[-1]
    #find the RElogfile containing the key word
    RElogfile = [x for x in RElogfiles if key in x][0]
    #pair the tifffile and RElogfile together
    pair = [tifffile, RElogfile]
    #append the pair to allfiles
    allfiles.append(pair)

In [26]:
#create a folder named UnrotTiff under the parent folder
UnrotTiffFolder = parent_folder + "UnrotTiff"
if not os.path.exists(UnrotTiffFolder):
    os.makedirs(UnrotTiffFolder)

#for each pair of tiff file and RElog file, unrotate the tiff file and save it into a folder named UnrotTiff
for pair in allfiles:
    tiff_file = pair[0]
    RElog_file = pair[1]
    #geterante the unrotated tiff file name
    unrotated_tiff_file = parent_folder+tiff_file.split("/")[-1].split(".")[0] + "_unrot.tif"
    #print processing file name
    print("Processing file: " + tiff_file.split("/")[-1])
  
    #unrotate the tiff file and save it into the UnrotTiff folder
    S = SITiffIO()
    S.open_tiff_file(tiff_file, "r")
    S.open_tiff_file(unrotated_tiff_file, "w")
    S.open_rotary_file(RElog_file)
    S.interp_times()  # might take a while...
    
    N = S.get_n_frames()
    Alltheta = S.get_all_theta()
    rotCenter = [rotx, roty]

    for i in range(N):
        frame_i = S.get_frame(i+1)
        theta_i = Alltheta[i]
        #unrotate the frame
        unrotatedFrame = Image.fromarray(frame_i).rotate(theta_i, center=rotCenter)
        #crop the largest inner rectangle from the unrotated frame
        croppedFrame = cropLargestRecT(unrotatedFrame, rotCenter)
        #convert the PIL image back to int16
        croppedFrame = np.array(croppedFrame, dtype=np.int16)
        S.write_frame(croppedFrame, i)


Processing file: 25072023_00003.tif
Loading rotary encoder file: /home/zilong/Desktop/2D2P/Data/183_25072023/REdata_20230725_155545.00003.txt
Calculating rotations and times from rotary encoder data...
Finished calculating rotations and times.
The rotary encoder file has 29392 timestamps in it.
Counted 9000 frames
Processing file: 25072023_00004.tif
Loading rotary encoder file: /home/zilong/Desktop/2D2P/Data/183_25072023/REdata_20230725_161010.00004.txt
Calculating rotations and times from rotary encoder data...
Finished calculating rotations and times.
The rotary encoder file has 18893 timestamps in it.
Counted 9000 frames
Processing file: 25072023_00005.tif
Loading rotary encoder file: /home/zilong/Desktop/2D2P/Data/183_25072023/REdata_20230725_162308.00005.txt
Calculating rotations and times from rotary encoder data...
Finished calculating rotations and times.
The rotary encoder file has 19375 timestamps in it.
Counted 9000 frames
Processing file: 25072023_00006.tif
Loading rotary e